# Large Language Models - Chatbots with Gradio

Gradio is a web app framework designed to facilitate the development and deployment of ML and DL apps. Have a look at [their website](https://www.gradio.app).

The following adapts their [Quickstart Guide](https://www.gradio.app/guides/quickstart).

Notebook by [Jérémie C. Wegner](https://jeremiewenger.com/about/).

In [ ]:
!pip install gradio

In [ ]:
import gradio as gr
import numpy as np

---

## 1. Intro

### Hello, World

Docs:

- [Textbox](https://www.gradio.app/docs/textbox)
- [Interface](https://www.gradio.app/docs/interface)

In [ ]:
def greet(name):
    return f"Hello {name}!"

demo = gr.Interface(
    fn=greet,
    inputs=gr.Textbox( # customize your textbox
        lines=2,
        placeholder="Name here..."
        ),
    outputs="text"
)

demo.launch()

### Multiple Input and Output Components

Docs:

- [Slider](https://www.gradio.app/docs/slider)

In [ ]:
import gradio as gr

# how about adding a second "checkbox" as a third input
# to allow the user to tick whether it's rainy or not,
# and add text that changes accordingly?
def greet(name, is_morning, temperature):
    salutation = "Good morning" if is_morning else "Good evening"
    greeting = f"{salutation} {name}. It is {temperature} degrees today"
    celsius = (temperature - 32) * 5 / 9
    return greeting, round(celsius, 2)

demo = gr.Interface(
    fn=greet,
    inputs=[
        "text",
        "checkbox",
        gr.Slider(0, 100) # you can add a default 'value=75' to your slider if you want
    ],
    outputs=[     # to add labels, try this (thx ChatGPT!):
        "text",   #gr.Textbox(label="Greeting"),  # Custom label for the first output
        "number"  #gr.Number(label="Temperature in Celsius")  # Custom label for the second output
    ]
)
demo.launch()


### An Image Example

You are obviously free to do whatever you like with your inputs, and they are not limited to text only! Here is an example where we modify an image.

Docs:
- [Image](https://www.gradio.app/docs/image)

In [ ]:
def filter(input_img):
    # sepia filter
    img_filter = np.array([
        [0.393, 0.769, 0.189],
        [0.349, 0.686, 0.168],
        [0.272, 0.534, 0.131]
    ])
    img_filter = img_filter.astype(np.float64) # make sure the contents are floats
    filter_img = input_img.dot(img_filter.T)
    filter_img /= filter_img.max()
    return filter_img

demo = gr.Interface(filter, gr.Image(), "image")

demo.launch(show_api=False, debug=True)

# There are quite a few matrices that have an interesting effect,
# it can be a nice idea to go look for others and modify this app
# to allow the user to choose between different filters!
# Color Inversion
# img_filter = np.array([
#     [-1, 0, 0],
#     [0, -1, 0],
#     [0, 0, -1]
# ]) + 1

# # Cool Filter
# img_filter = np.array([
#     [0.9, 0, 0],
#     [0, 0.9, 0],
#     [0, 0, 1.1]
# ])

### Using Blocks

For more control, `Blocks` are the way to go. Here we can play with flipping various kinds of data left/right or upside down!

Docs:
- [Blocks](https://www.gradio.app/docs/blocks)
- [Markdown](https://www.gradio.app/docs/markdown)
- [Tab](https://www.gradio.app/docs/tab)
- [Button](https://www.gradio.app/docs/button)
- [Accordion](https://www.gradio.app/docs/accordion)
- [Audio](https://www.gradio.app/docs/audio)

Also [`np.fliplr`](https://numpy.org/doc/stable/reference/generated/numpy.fliplr.html) and [`np.flipud`](https://numpy.org/doc/stable/reference/generated/numpy.flipud.html).

In [ ]:
def flip_text(x):
    return x[::-1]

def flip_image(x):
    return np.fliplr(x)  # try also np.flipud

# help from ChatGPT (buggy, but still nice) for this!
def reverse_audio(audio_data):
    # audio_data is a tuple with (sample_rate, audio_array)
    # print(audio_data)
    sample_rate, audio_array = audio_data
    reversed_audio = audio_array[::-1]  # Reverse the audio data
    # Return the reversed audio data along with the sample rate
    return (sample_rate, reversed_audio)

# note the 'with' syntax, to allow you to populate your blocks
with gr.Blocks() as demo:
    # use markdown syntax in the app
    gr.Markdown("# Flip text, image, or audio files using this demo.")
    with gr.Tab("Flip Text"):
        text_input = gr.Textbox()
        text_output = gr.Textbox()
        text_button = gr.Button("Flip")
    with gr.Tab("Flip Image"):
        with gr.Row():
            image_input = gr.Image()
            image_output = gr.Image()
        image_button = gr.Button("Flip")
    with gr.Tab("Reverse Audio"):
        with gr.Row():
            audio_input = gr.Audio()
            audio_output = gr.Audio()
        audio_button = gr.Button("Reverse")

    with gr.Accordion("Open for More!"):
        gr.Markdown("Look at me...")

    # now we have three different functions, with three different effects, in each one of our tabs!
    text_button.click(flip_text, inputs=text_input, outputs=text_output)
    image_button.click(flip_image, inputs=image_input, outputs=image_output)
    audio_button.click(reverse_audio, inputs=audio_input, outputs=audio_output)

demo.launch(show_api=False, debug=True)

---

## 2. Chatbots!

Adapated from  the [Chatbot](https://www.gradio.app/guides/quickstart#chatbots) part of the Quickstart, [How to Create a Chatbot with Gradio](https://www.gradio.app/guides/creating-a-chatbot-fast#a-streaming-example-using-openai) and [How to Create a Custom Chatbot with Gradio Blocks](https://www.gradio.app/guides/creating-a-custom-chatbot-with-blocks#adding-markdown-images-audio-or-videos).

Gradio apps these days are used for two main purposes:
- to show off diffusion models (we are doing this too!);
- to build chatbots.

The [triple threat](https://en.wiktionary.org/wiki/triple_threat) of Huggingface and its Hub of models/tokenizers/datasets, Gradio apps and [Huggingface Spaces](https://huggingface.co/spaces) (to deploy apps and provide GPUS) had made this open-source corp hugely important in the field, and turbocharged the development of this ecosystem.

Let's look at some chatbot examples.

#### Check out [Huggingface Spaces](https://huggingface.co/spaces)!

That works like GitHub in that you create repository following a certain syntax for your app, and then you can edit the app locally, push it to Spaces and see the updates (even if you don't have a GPU on your machine). And just like on GitHub, you can *fork* projects (copy and import someone else's code into your own account and then modify it to make it your own). Check out the [intro guide](https://huggingface.co/spaces/launch).

In [ ]:
import sys

if 'google.colab' in sys.modules:
    !pip -q install gradio
    !pip -q install transformers

import gradio as gr

In [ ]:
import time
import random

### Example: a chatbot that responds yes or no

This is obviously the simplest possible thing you can do, but this logic of randomness can be combined with branches (`if/else`) to create all sorts of elaborate paths of decision guiding your bot's answers: attempts at chatbots before Deep Learning would use elaborate systems like this one (for the literature lovers, see [this](https://drive.google.com/file/d/1v-q2M8ZlCcoCGKvw0rgpmyBg6bbH4DWF/view?usp=sharing), [that](https://drive.google.com/file/d/12-wIbonK8d8w5UXpFM_-nzMfBoDpq4Zq/view?usp=sharing) and [that](https://drive.google.com/file/d/1G_T2MjCQCuLYIQPpe0GnBOu3PO67deyL/view?usp=sharing) articles, for instance).

Docs:
- [ChatInterface](https://www.gradio.app/docs/chatinterface)

And the [`random.choice`](https://docs.python.org/3/library/random.html#functions-for-sequences) function in Python.

In [ ]:
def random_response(message, history):
    return random.choice(["Yes", "No"])

gr.ChatInterface(random_response).launch()

### Another example using the user’s input and history

Here you can see that I use both a modulo logic (a bit boring, randomness might be nicer) to decide between three different kinds of answer. Then, in option 2, I randomly pick one interaction in the history, and use the user input!

Reminder: `history` is mentioned [here](https://www.gradio.app/guides/quickstart#chatbots).


In [ ]:
def sometimes_agree_sometimes_remembers(message, history):
    # print(*history, sep="\n") # you can have a look at the history object if you want
    if len(history) % 3 == 0:
        return f"Yes, I do think that '{message}'"
    elif len(history) % 3 == 1:
        # history is an array of arrays, each containing ["user input", "bot response"]
        past_message = random.choice(history)[0] # [0] for user input, [1] for bot response
        return f"Wait, didn't you say earlier: '{past_message}'"
    else:
        return "I don't think so"

gr.ChatInterface(sometimes_agree_sometimes_remembers).launch(debug=True)

### Streaming chatbots

Isn't it nice if your bot types live instead of giving you a full answer?

For that, and any gradual process of answering, use the [`yield` keyword of Python generators](https://realpython.com/introduction-to-python-generators/).

See [this part](https://www.gradio.app/guides/creating-a-chatbot-fast#a-streaming-example-using-openai).

In [ ]:
def slow_echo(message, history):
    for i in range(len(message)):
        time.sleep(0.3) # try random.random() * .5 (or another number) for irregular typing speed!
        yield f"You typed: {message[: i+1]}"

gr.ChatInterface(slow_echo).queue().launch()

---

## Experiments

- You could try to modify this code to work with the latest Llama models by Meta (you must register on [their site](https://ai.meta.com/llama/), then on Huggingface once you get permission, to be able to download the code). After that (same as with various restricted models/datasets/etc. on the Hub), you would need to log into HF:
```python
from pathlib import Path
from huggingface_hub import notebook_login
if not (Path.home()/'.huggingface'/'token').exists():
    notebook_login()
```
- Another example that would allow you to play with the cutting-edge LLMs is the [OpenAI example](https://www.gradio.app/guides/creating-a-chatbot-fast#a-streaming-example-using-openai) in the Gradio tutorial. You would first need to register (with credit card) and get an API key on [their website](https://platform.openai.com/)...

- Gradio ships with a [`Flagging`](https://www.gradio.app/guides/key-features#styling) logic, that allows you to harvest data from your users for free! You can also implement [`likes`](https://www.gradio.app/guides/creating-a-custom-chatbot-with-blocks#liking-disliking-chat-messages), that could be interesting!

- The current trend these days is to work with multimodality (systems that are able to handle more than one type of data: text and images, for instance, or text and music). See [this last part](https://www.gradio.app/guides/creating-a-custom-chatbot-with-blocks#adding-markdown-images-audio-or-videos) of the Gradio Chatbot tutorial for examples, as well as the two apps they recommend [project-baize/Baize-7B](https://huggingface.co/spaces/project-baize/chat-with-baize) and [MAGAer13/mPLUG-Owl](https://huggingface.co/spaces/MAGAer13/mPLUG-Owl) (and as said you could clone these projects, study the code, and transform them into your own project)!
